In [24]:
import pickle
import numpy as np
import datasets
from datasets import load_dataset, Dataset
import pyarrow as pa
import pandas as pd


from dotenv import load_dotenv
load_dotenv()

import nltk
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of translate failed: Traceback (most recent call last):
  File "/Users/juglardiaz-zamora/Projects/multilingual-augmented-llm/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/juglardiaz-zamora/Projects/multilingual-augmented-llm/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/juglardiaz-zamora/Projects/multilingual-augmented-llm/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/juglardiaz-zamora/Projects/multilingual-augmented-llm/.venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 309, in update_function
    setattr(old, name, getattr(new, name))
ValueError: translate_text() requires a code object with 0 free vars, not 2
]


In [25]:
import sys
sys.path.append("../..")
from translate import translate_text

In [3]:
TASKS = [
        'abstract_algebra',
        'anatomy',
        'astronomy',
        'business_ethics',
        'clinical_knowledge',
        'college_biology',
        'college_chemistry',
        'college_computer_science',
        'college_mathematics',
        'college_medicine',
        'college_physics',
        'computer_security',
        'conceptual_physics',
        'econometrics',
        'electrical_engineering',
        'elementary_mathematics',
        'formal_logic',
        'global_facts',
        'high_school_biology',
        'high_school_chemistry',
        'high_school_computer_science',
        'high_school_european_history',
        'high_school_geography',
        'high_school_government_and_politics',
        'high_school_macroeconomics',
        'high_school_mathematics',
        'high_school_microeconomics',
        'high_school_physics',
        'high_school_psychology',
        'high_school_statistics',
        'high_school_us_history',
        'high_school_world_history',
        'human_aging',
        'human_sexuality',
        'international_law',
        'jurisprudence',
        'logical_fallacies',
        'machine_learning',
        'management',
        'marketing',
        'medical_genetics',
        'miscellaneous',
        'moral_disputes',
        'moral_scenarios',
        'nutrition',
        'philosophy',
        'prehistory',
        'professional_accounting',
        'professional_law',
        'professional_medicine',
        'professional_psychology',
        'public_relations',
        'security_studies',
        'sociology',
        'us_foreign_policy',
        'virology',
        'world_religions']

In [4]:
len(TASKS)

57

In [7]:
first_load = False
data = dict()
if first_load:
    for task in TASKS:
        data[task] = load_dataset("lukaemon/mmlu", task)

else:
    pass
    with open('data/multi_mmlu.pickle', 'rb') as handle:
        data = pickle.load(handle)

In [8]:
data['abstract_algebra'].keys()

dict_keys(['test', 'validation', 'train', 'test_es_gt'])

In [9]:
data['abstract_algebra']['test'][0]


{'input': 'Let p = (1, 2, 5, 4)(2, 3) in S_5 . Find the index of <p> in S_5.',
 'A': '8',
 'B': '2',
 'C': '24',
 'D': '120',
 'target': 'C'}

In [10]:
data['abstract_algebra']['test_es_gt'][0]


{'entrada': 'Sea p = (1, 2, 5, 4)(2, 3) en S_5. Encuentre el índice de <p> en S_5.',
 'A': '8',
 'B': '2',
 'C': '24',
 'D': '120',
 'objetivo': 'C'}

In [9]:
len(data['abstract_algebra']['test'])

99

In [11]:
sum([len(data[task]['test']) for task in TASKS])

13985

In [8]:
data['abstract_algebra']['test'][1]


{'input': 'Find all zeros in the indicated finite field of the given polynomial with coefficients in that field. x^5 + 3x^3 + x^2 + 2x in Z_5',
 'A': '0',
 'B': '1',
 'C': '0,1',
 'D': '0,4',
 'target': 'D'}

In [7]:
data['abstract_algebra']['train'][0]


{'input': 'Statement 1 | If aH is an element of a factor group, then |aH| divides |a|. Statement 2 | If H and K are subgroups of G then HK is a subgroup of G.',
 'A': 'True, True',
 'B': 'False, False',
 'C': 'True, False',
 'D': 'False, True',
 'target': 'B'}

In [14]:
data['abstract_algebra']['train'][0].keys()

dict_keys(['input', 'A', 'B', 'C', 'D', 'target'])

In [12]:
data['abstract_algebra']['train'][0]['input']

'Statement 1 | If aH is an element of a factor group, then |aH| divides |a|. Statement 2 | If H and K are subgroups of G then HK is a subgroup of G.'

In [13]:
data['abstract_algebra']['train'][0]['target']


'B'

In [12]:
mem = {
    'es': {'input': 'entrada', 'True': 'Verdadero',  'False': 'Falso', 'target': 'objetivo'}, 
    'pt': {'input': 'entrada', 'True': 'Verdadeiro', 'False': 'Falso', 'target': 'alvo'},
    'fr': {'input': 'entrée',  'True': 'Vrai',       'False': 'Faux',  'target': 'cible'}, 
    'it': {'input': 'input',   'True': 'Vero',       'False': 'Falso', 'target': 'obiettivo'}, 
}


In [22]:
#what is the input to the algorithm
#what is the target solution
#is it true or false

def translate_text_aux(target='', text=''):
    if text.isnumeric():
        return text
    response = translate_text(target=target, text=text)
    return response["translatedText"]

def translate_example(example, lang_cod):
    new_example = {}
    for key in example.keys():
        if key in mem[lang_cod]:
            new_key = mem[lang_cod][key]
        else:
            new_key = translate_text_aux(target=lang_cod, text=key)

        mem[lang_cod][key] = new_key

        value = example[key]

        if value in mem[lang_cod]:
            new_value = mem[lang_cod][value]
        else:
            new_value = translate_text_aux(target=lang_cod, text=value)

        mem[lang_cod][value] = new_value
        
        new_example[new_key] = new_value
    return new_example

def run_translate_task(lang_cod, task, num_examples=0):
    if not num_examples:
        num_examples = len(data[task]['test'])
    ### call translate function
    data[task][f"test_{lang_cod}_gt"] = [translate_example(data[task]['test'][i], lang_cod) for i in range(num_examples)]
    
    ### convert to Huggingface dataset
    data[task][f"test_{lang_cod}_gt"] = Dataset(pa.Table.from_pandas(pd.DataFrame(data[task][f"test_{lang_cod}_gt"])))

    ### save to disk
    with open('data/multi_mmlu.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
len(mem['es'])

58704

## Spanish

In [41]:
for task in TASKS:
    print(task)
    run_translate_task('es', task)
    

abstract_algebra
anatomy
astronomy
business_ethics
clinical_knowledge
college_biology
college_chemistry
college_computer_science
college_mathematics
college_medicine
college_physics
computer_security
conceptual_physics
econometrics
electrical_engineering
elementary_mathematics
formal_logic
global_facts
high_school_biology
high_school_chemistry
high_school_computer_science
high_school_european_history
high_school_geography
high_school_government_and_politics
high_school_macroeconomics
high_school_mathematics
high_school_microeconomics
high_school_physics
high_school_psychology
high_school_statistics
high_school_us_history
high_school_world_history
human_aging
human_sexuality
international_law
jurisprudence
logical_fallacies
machine_learning
management
marketing
medical_genetics
miscellaneous
moral_disputes
moral_scenarios
nutrition
philosophy
prehistory
professional_accounting
professional_law
professional_medicine
professional_psychology
public_relations
security_studies
sociology
us_f

In [23]:
for task in TASKS:
    print(task)
    run_translate_task('pt', task)
    

abstract_algebra
anatomy
astronomy
business_ethics
clinical_knowledge
college_biology
college_chemistry
college_computer_science
college_mathematics
college_medicine
college_physics
computer_security
conceptual_physics
econometrics
electrical_engineering
elementary_mathematics
formal_logic
global_facts
high_school_biology
high_school_chemistry


ReadTimeout: HTTPSConnectionPool(host='translation.googleapis.com', port=443): Read timed out. (read timeout=60)

In [26]:
for task in TASKS[19:]:
    print(task)
    run_translate_task('pt', task)

high_school_chemistry
high_school_computer_science
high_school_european_history
high_school_geography
high_school_government_and_politics
high_school_macroeconomics
high_school_mathematics
high_school_microeconomics
high_school_physics
high_school_psychology
high_school_statistics
high_school_us_history
high_school_world_history
human_aging
human_sexuality
international_law
jurisprudence
logical_fallacies
machine_learning
management
marketing
medical_genetics
miscellaneous
moral_disputes
moral_scenarios
nutrition
philosophy
prehistory
professional_accounting
professional_law
professional_medicine
professional_psychology
public_relations
security_studies
sociology
us_foreign_policy
virology
world_religions


In [28]:
len(data['abstract_algebra']['test_pt_gt'])

99

In [29]:
len(data['world_religions']['test_pt_gt'])


170

In [30]:
len(data['high_school_biology']['test_pt_gt'])


309